In [3]:
from google.colab import files
import pandas as pd

# Upload de múltiplos arquivos
uploaded = files.upload()

# Criar lista de DataFrames a partir de cada arquivo parquet
dfs = []
for file_name in uploaded.keys():
    df_temp = pd.read_parquet(file_name)
    dfs.append(df_temp)

# Concatenar todos em um único DataFrame
df = pd.concat(dfs, ignore_index=True)

# Visualizar
df.head()



Saving part-00000-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet to part-00000-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet
Saving part-00001-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet to part-00001-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet
Saving part-00002-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet to part-00002-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet
Saving part-00003-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet to part-00003-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet
Saving part-00004-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet to part-00004-807aec45-9e7c-4d19-9bdc-06c9096c6b28-c000.snappy.parquet


,ID,Comentarios
0,1419,Está muito caro e não tem mais a mesma qualida...
1,1419,"Eu adoro este tapete, ele não deixa cheiro e a..."
2,1419,Absorve bem. Indico.
3,1419,Já usei vários tapetes higiênicos mas o super ...
4,1419,"Excelente qualidade, oi transporte sempre foi ..."


In [4]:
df.shape

(40842, 2)

In [ ]:
# 1. Instalar transformers (caso não esteja instalado)
!pip install -q transformers
!pip install -q spacy transformers
!python -m spacy download pt_core_news_sm

# 2. Importar bibliotecas
from transformers import pipeline
import numpy as np
import spacy

# Carrega modelo SpaCy para português
nlp = spacy.load("pt_core_news_sm")

# Pipeline de sentimento com BERT multilíngue
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 4. Carregar pipeline de análise de sentimentos
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")



In [ ]:
# 5. Função para mapear o resultado em classes simplificadas
def analisar_sentimento(texto):
    try:
        resultado = classifier(texto[:512])[0]  # Truncar para 512 tokens (limite do BERT)
        estrelas = int(resultado['label'][0])  # Ex: '5 stars' → 5
        if estrelas <= 2:
            return "negativo"
        elif estrelas == 3:
            return "neutro"
        else:
            return "positivo"
    except:
        return "erro"


def extrair_adjetivo_principal(texto):
    doc = nlp(texto.lower())
    adjetivos = [token for token in doc if token.pos_ == "ADJ"]
    if not adjetivos:
        return None
    return adjetivos[0].lemma_

def classificar_sentimento(palavra):
    if not palavra:
        return "neutro"
    try:
        resultado = classifier(palavra[:512])[0]
        estrelas = int(resultado['label'][0])
        if estrelas <= 2:
            return "negativo"
        elif estrelas == 3:
            return "neutro"
        else:
            return "positivo"
    except:
        return "neutro"


In [ ]:
# 6. Aplicar ao DataFrame (pode demorar dependendo do volume)
df["sentimento"] = df["Comentarios"].astype(str).apply(analisar_sentimento)
df["Palavra_Chave"] = df["Comentarios"].astype(str).apply(extrair_adjetivo_principal)
df["percepcao"] = df["Palavra_Chave"].apply(classificar_sentimento)


In [ ]:
df.head()

,ID,Comentarios,sentimento,Palavra_Chave,percepcao
0,1419,Está muito caro e não tem mais a mesma qualida...,negativo,caro,neutro
1,1419,"Eu adoro este tapete, ele não deixa cheiro e a...",positivo,None,neutro
2,1419,Absorve bem. Indico.,positivo,indico,neutro
3,1419,Já usei vários tapetes higiênicos mas o super ...,positivo,higiênico,positivo
4,1419,"Excelente qualidade, oi transporte sempre foi ...",positivo,excelente,positivo


In [ ]:
df.to_parquet('sentimentos.parquet', index=False)

In [ ]:
files.download('sentimentos.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>